# Predicting Impact of Climate on Polar Bear Habitat Selection

## Dataset from 1985 - 2016 

#### Machine Learning Pipeline Overview
1. Import and clean the dataset -- remove null values, select relevant columns, remove outliers
2. Convert RSF values to categorical values
3. Assign features and target values then split data into training and testing datasets
4. Normalize quantitative data using MinMaxScalar
4. Define, Train and Score Models
5. Select highest scoring model and hypertune (Random Forest)
6. Save model using pickle for future predictions

In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /opt/anaconda3/lib/python3.7/site-packages (0.0)


In [2]:
# install joblib for saving
# Restart kernel after installing 
!pip install joblib

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
# import missingno as msno
import seaborn as sns

In [4]:
df = pd.read_csv("../../../data/ml_compile.csv")

In [5]:
df.describe()

,land_avg_temp,land_max_temp,land_min_temp,land_ocean_avg_temp,north_min_temp_anomoly,north_max_temp_anomoly,north_mean_temp_anomoly,global_avg_co2,seaice_extent,bear_rsf_mean,...,latitude_rsf,loc_zone,num_bears,avg_distance_traveled,avg_num_land_dens,avg_num_ice_dens,avg_num_active_dens,den_lat,den_long,den_loc
count,1392.000000,1392.000000,1392.000000,1392.000000,1428.000000,1434.000000,1446.000000,752.000000,506.000000,334.000000,...,334.000000,334.000000,84.000000,8.400000e+01,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000
mean,8.762454,14.516995,3.000047,15.317030,0.122602,0.213536,0.213620,355.582926,11.405146,15.820979,...,71.788726,3.886228,6.690476,5.916332e+08,41.553384,504.157876,40.258267,-147.752653,72.663690,4.404762
std,4.199958,4.294911,4.114758,1.259405,0.709268,0.620087,0.630979,28.949294,3.252910,2.430975,...,1.610578,2.351714,3.951441,6.769937e+08,76.629269,481.543071,70.866545,9.706844,1.605944,2.657651
min,1.395000,7.082000,-4.298000,12.839000,-1.951000,-1.772000,-1.647000,312.430000,3.565600,6.093212,...,67.796405,0.000000,1.000000,1.784230e+05,0.000000,0.000000,0.000000,-165.942373,70.318495,0.000000
25%,4.655750,10.426250,-1.103250,14.150750,-0.354250,-0.184750,-0.198000,329.112500,8.571152,14.872230,...,70.898015,2.000000,3.000000,1.296206e+08,0.000000,171.946429,0.000000,-154.116038,71.463159,2.000000
50%,9.087500,14.899000,3.213500,15.373000,0.022500,0.140500,0.098000,352.880000,12.095933,16.388356,...,71.314351,4.000000,7.000000,2.588397e+08,10.083333,270.750000,0.000000,-149.928811,72.200680,5.000000
75%,12.953000,18.867000,7.048000,16.477750,0.583250,0.563500,0.576500,378.857500,14.318582,17.471275,...,72.192211,5.000000,8.000000,9.286488e+08,55.937500,820.386905,68.392857,-143.258156,73.719433,7.000000
max,15.482000,21.320000,9.715000,17.611000,2.970000,2.805000,2.877000,417.070000,16.341938,20.000000,...,78.895838,9.000000,17.000000,2.617030e+09,422.333333,2012.500000,333.000000,-120.994143,76.812461,9.000000


In [6]:
# df = df[['land_avg_temp', 'land_max_temp', 'land_min_temp',
#        'land_ocean_avg_temp', 'north_min_temp_anomoly',
#        'north_max_temp_anomoly', 'north_mean_temp_anomoly', 'global_avg_co2',
#        'seaice_extent', 'bear_rsf_mean']]
df = df[['land_avg_temp', 'land_ocean_avg_temp','global_avg_co2',
       'seaice_extent', 'bear_rsf_mean']]

In [7]:
df = df.dropna(axis=0, how="any")
df.describe()

,land_avg_temp,land_ocean_avg_temp,global_avg_co2,seaice_extent,bear_rsf_mean
count,331.000000,331.000000,331.000000,331.000000,331.000000
mean,9.385082,15.712589,372.225619,11.357481,15.851405
std,4.121994,1.225211,16.649756,3.220250,2.392539
min,2.710000,13.566000,343.130000,3.565600,6.093212
25%,5.552000,14.511000,356.145000,8.513177,14.928223
50%,9.618000,15.811000,372.250000,12.016400,16.396830
75%,13.443000,16.884000,386.235000,14.326682,17.470912
max,15.482000,17.611000,403.960000,16.050143,20.000000


In [8]:
from statistics import median 
print(median(df["bear_rsf_mean"]))

16.39683045


## Categorical Algorithms
### Preprocessing - Transform RSF values to Categorical

In [9]:
conditions = [(df["bear_rsf_mean"] < 15), (df["bear_rsf_mean"] > 15) & (df["bear_rsf_mean"] < 18), (df["bear_rsf_mean"]>18)]
values = ['low', 'mid', 'high']
df["bear_mean_cat"] = np.select(conditions,values)

In [11]:
#Change Threshold to 9
df_cat = df [['land_avg_temp', 'land_ocean_avg_temp', 'global_avg_co2',
       'seaice_extent','bear_mean_cat']]

### Assign new variables

In [12]:
X = df_cat[['land_avg_temp', 'land_ocean_avg_temp', 'global_avg_co2',
       'seaice_extent']]
y_cat = df_cat["bear_mean_cat"]
print(X.shape, y_cat.shape)

(331, 4) (331,)


### Split the data into testing and training dataset

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_cat_train, y_cat_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

### MinMaxScalar to fit and transform X features and y target

In [14]:
#Fit Transform using MinMaxScalar for X features
from sklearn.preprocessing import MinMaxScaler
X_minmax = MinMaxScaler().fit(X_train)

X_train_minmax = X_minmax.transform(X_train)
X_test_minmax = X_minmax.transform(X_test)

# Random Forest Analysis

In [15]:
from sklearn.ensemble import RandomForestClassifier
y_cat_rf = df_cat["bear_mean_cat"]
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_minmax, y_cat_train)
score = rf.score(X_test_minmax, y_cat_test)

In [16]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y_cat_train)
encoded_y_train = label_encoder.transform(y_cat_train)
encoded_y_test = label_encoder.transform(y_cat_test)

In [17]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_cat_train)
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

{'high': 0, 'low': 1, 'mid': 2}


In [18]:
rf2 = RandomForestClassifier(n_estimators=200, min_samples_split =5, min_samples_leaf = 4, max_features='auto', max_depth=60, bootstrap='True')
rf2 = rf2.fit(X_train_minmax, encoded_y_train)
score = rf2.score(X_test_minmax, encoded_y_test)
score

0.6268656716417911

In [21]:
import pickle
from pickle import dump
pickle.dump(rf2, open("../../models/rf_rsf2.pkl", "wb"))

In [22]:
#load_file = open("rf_rsf.pkl", "rb")
with open("../../models/rf_rsf2.pkl", "rb") as f:
    model_object = pickle.load(f)
    f.close()

In [23]:
model_object

RandomForestClassifier(bootstrap='True', ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=60, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)